# Tarefa 02 Módulo 05

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito. 
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu. 

In [40]:
import pandas as pd
import numpy as np

In [42]:
propostas = pd.read_csv('application_record.csv')
# propostas = pd.read_csv('https://github.com/diegofurukawa/Python_Ebac/blob/54208d0f746d7a06c24f3a3d93caadbe8bcb4e14/Profissao_Cientista_de_Dados/02_Cientista_de_Dados/Modulo_05_-_Limpeza_e_preparacao_de_dados/exercicio_2/application_record.csv')

pg = pd.read_csv('pagamentos_largo.csv')
# pg = pd.read_csv('https://github.com/diegofurukawa/Python_Ebac/blob/54208d0f746d7a06c24f3a3d93caadbe8bcb4e14/Profissao_Cientista_de_Dados/02_Cientista_de_Dados/Modulo_05_-_Limpeza_e_preparacao_de_dados/exercicio_2/pagamentos_largo.csv')

In [43]:
propostas = propostas.set_index('ID')

In [46]:
propostas.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
ID,,,,,,,,,,,,,,,,,
5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [48]:
pg = pg.set_index('ID')

In [50]:
# 1) Seu código aqui

pg.isin(['0'])

,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
ID,,,,,,,,,,,,,,,,,,,,,
5001718,True,True,True,True,True,True,True,True,False,True,...,False,True,False,True,True,True,True,False,True,False
5001719,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5001720,True,True,True,True,True,True,True,True,True,True,...,False,True,True,True,True,True,True,True,True,True
5001723,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,True,True,False,False
5001726,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150475,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5150476,True,True,True,True,True,True,True,True,True,False,...,False,False,False,True,True,True,True,True,True,True
5150480,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,False,False


In [14]:
pg.columns

Index(['mes_0', 'mes_1', 'mes_10', 'mes_11', 'mes_12', 'mes_13', 'mes_14',
       'mes_15', 'mes_16', 'mes_17', 'mes_18', 'mes_19', 'mes_2', 'mes_20',
       'mes_21', 'mes_22', 'mes_23', 'mes_24', 'mes_3', 'mes_4', 'mes_5',
       'mes_6', 'mes_7', 'mes_8', 'mes_9'],
      dtype='object')

In [56]:
# 2) Seu código aqui

pgnew = pg.isin(['0', '1'])
pgnew = pgnew.groupby('ID').sum()

# replace string value with NaN
pgnew = pgnew.replace(0, np.nan, regex=True)

pgbonspagadores = pgnew[['mes_0', 'mes_1', 'mes_10', 'mes_11', 'mes_12', 'mes_13', 'mes_14',
       'mes_15', 'mes_16', 'mes_17', 'mes_18', 'mes_19', 'mes_2', 'mes_20',
       'mes_21', 'mes_22', 'mes_23', 'mes_24', 'mes_3', 'mes_4', 'mes_5',
       'mes_6', 'mes_7', 'mes_8', 'mes_9']].dropna()

# Realizando de Reset do Index
pgbonspagadores = pgbonspagadores.reset_index()

# Aplicando ID como Index do DF
pgbonspagadores = pgbonspagadores.set_index('ID')

display(pgbonspagadores)



,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,mes_17,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
ID,,,,,,,,,,,,,,,,,,,,,
5001720,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5001793,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5001824,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5001838,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5001857,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150052,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5150067,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5150229,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [84]:
# 3) Seu código aqui

# pd.concat([propostas, pgbonspagadores])

propostas_result = pd.merge(propostas, pgbonspagadores, on='ID')

display(propostas_result)






,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
ID,,,,,,,,,,,,,,,,,,,,,
5008825,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5008838,M,N,Y,1,405000.0,Commercial associate,Higher education,Married,House / apartment,-11842,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5008844,M,Y,Y,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-20502,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5008847,M,Y,Y,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-20502,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5008850,M,Y,Y,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-20502,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150004,M,Y,Y,1,225000.0,Working,Higher education,Separated,Municipal apartment,-17205,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5150047,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5150052,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [86]:
# 4) eu código aqui

display(propostas_result)


,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
ID,,,,,,,,,,,,,,,,,,,,,
5008825,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5008838,M,N,Y,1,405000.0,Commercial associate,Higher education,Married,House / apartment,-11842,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5008844,M,Y,Y,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-20502,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5008847,M,Y,Y,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-20502,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5008850,M,Y,Y,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-20502,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150004,M,Y,Y,1,225000.0,Working,Higher education,Separated,Municipal apartment,-17205,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5150047,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5150052,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
# 4) eu código aqui

